In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

dataset_name = 'cats_vs_dogs'
# train_dataset = tfds.load(name=dataset_name, split='train')
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

def preprocess(data):
    x = data['image']
    y = data['label']
    x = x / 255
    x = tf.image.resize(x, size=(224,224))
    return x, y

def solution_model():
    batch_size = 32
    train_data = train_dataset.map(preprocess).batch(batch_size)
    valid_data = valid_dataset.map(preprocess).batch(batch_size)

    transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 
    transfer_model.trainable=False

    model = Sequential([
                        # Conv2D(64, (3,3), input_shape = (224,224,3), activation='relu'),
                        # MaxPooling2D(2,2),
                        # Conv2D(64, (3,3), activation='relu'),
                        # MaxPooling2D(2,2),
                        # Conv2D(128, (3,3), activation='relu'),
                        # MaxPooling2D(2,2),
                        # Conv2D(128, (3,3), activation='relu'),
                        # MaxPooling2D(2,2),
                        # Conv2D(256, (3,3), activation='relu'),
                        # MaxPooling2D(2,2),
                        transfer_model,
                        Flatten(),
                        Dropout(0.5),
                        Dense(512, activation='relu'),
                        Dense(128, activation='relu'),
                        Dense(2, activation='softmax'),
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

    checkpoint_path = 'my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                 save_weights_only = True,
                                 save_best_only = True,
                                 monitor = 'val_loss',
                                 verbose = 1)
    
    model.fit(train_data,
              validation_data = (valid_data),
              epochs = 20,
              callbacks = [checkpoint])
    
    model.load_weights(checkpoint_path)
    return model

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteMCEQTP/cats_vs_dogs-train.tfrecord


  0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [ ]:
model = solution_model()
model.save('(3) cats_dogs_tfds.h5')

58900480/58889256 [==============================] - 0s 0us/step
Epoch 1/20
582/582 [==============================] - 134s 169ms/step - loss: 0.3132 - acc: 0.8840 - val_loss: 0.1831 - val_acc: 0.9215

Epoch 00001: val_loss improved from inf to 0.18314, saving model to my_checkpoint.ckpt
Epoch 2/20
582/582 [==============================] - 100s 172ms/step - loss: 0.1812 - acc: 0.9252 - val_loss: 0.1622 - val_acc: 0.9304

Epoch 00002: val_loss improved from 0.18314 to 0.16216, saving model to my_checkpoint.ckpt
Epoch 3/20
582/582 [==============================] - 104s 179ms/step - loss: 0.1588 - acc: 0.9344 - val_loss: 0.1719 - val_acc: 0.9261

Epoch 00003: val_loss did not improve from 0.16216
Epoch 4/20
582/582 [==============================] - 103s 178ms/step - loss: 0.1416 - acc: 0.9412 - val_loss: 0.1682 - val_acc: 0.9327

Epoch 00004: val_loss did not improve from 0.16216
Epoch 5/20
582/582 [==============================] - 104s 178ms/step - loss: 0.1306 - acc: 0.9468 - val_lo